In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

1. Giriş
2. Kütüphaneler
3. Veri okuma - manuel veri hikayesi
4. Veriyi anlama
5. 3 den 8 e kadar olan kategorileri iyi ve kotü diye ayırma
6. Labelencoder ile etiketli sayısallaştırma
7. Özellik ölçeklendirme-Standarscaler
8. Model Kurma 
9. Modeli compile ve fit etme
10. Modeli değerlendirme
11. Tamamen yalıtılmış test verisi üzerinden tahmin etme

# **Kullanılan kütüphaneler**

In [ ]:
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, normalize, StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, r2_score
from sklearn.svm import SVC

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam, RMSprop
from keras.losses import sparse_categorical_crossentropy, binary_crossentropy
from keras.metrics import Accuracy
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras import regularizers
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.utils import np_utils, to_categorical
from keras.callbacks import ModelCheckpoint
import keras
from keras.callbacks import EarlyStopping

# **Veri okunması**

Daha öncesinden jupyter notebook ile train_test_split fonksiyonu kullanarak ve devamında to_csv diye dönüştürerek indirdiğimiz veri setini kaggle yükledik.

In [ ]:
data = pd.read_csv("../input/red-wine-manuel/train_manuel.csv", engine='python')
test_df= pd.read_csv("../input/red-wine-manuel/test_manuel.csv", engine='python')


# **Veriyi Anlama**

In [ ]:
data.isnull().sum()

In [ ]:
data.corr()["quality"].sort_values()

In [ ]:
df_train_corr=data.corr()
df_train_corr.style.background_gradient(cmap='coolwarm', axis=None)

In [ ]:
y = data['quality']
y_test = test_df['quality']


In [ ]:
y_frame=pd.DataFrame()
y_frame['kalite seviyesi']=y
y_frame.groupby(['kalite seviyesi']).size().plot.bar(figsize=(8,4),
                                                     title="Herbir Kalite Seviyesinin Eğitim Kümesindeki Dağılımı")
plt.show()

In [ ]:
y_frame=pd.DataFrame()
y_frame['kalite seviyesi']=y_test
y_frame.groupby(['kalite seviyesi']).size().plot.bar(figsize=(8,4),title="Herbir Kalite Seviyesinin Test Kümesindeki Dağılımı")
plt.show()

In [ ]:
#Özelliklerin kalite puanları ile ilişkisini göstermek için kullanılacak
#çizim türleri
def draw_multivarient_plot(dataset, rows, cols, plot_type):
    """
    dataset: Veri seti
    rows: Satır sayısı
    cols: sütün sayısı
    plot_type: Çizdirilecek grafik türü
    """
    
    #Veri setindeki sütünların isimleri alınıyor
    column_names=dataset.columns.values
    #Kaç tane sütün olduğu bulunuyor
    number_of_column=len(column_names)
    
    #Satır*sütün boyutlarında alt grafik içeren
    #matris oluşturuluyor. Matrisin genişliği:22 yüksekliği:16
    fig, axarr=plt.subplots(rows,cols, figsize=(22,16))

    counter=0# Çizimi yapılacak özelliğin column_names listesindeki indeks değerini tutuyor
    for i in range(rows):
        for j in range(cols):
            """
            i: satır numarasını tutuyor
            j: sütün numarasını tutuyor
            axarr[i][j]: Çizilen grafigin grafik matrisindeki yerini belirliyor
            """
            if 'violin' in plot_type:
                sns.violinplot(x='quality', y=column_names[counter],data=dataset, ax=axarr[i][j])
            elif 'box'in plot_type :
                sns.boxplot(x='quality', y=column_names[counter],data=dataset, ax=axarr[i][j])
            elif 'point' in plot_type:
                sns.pointplot(x='quality',y=column_names[counter],data=dataset, ax=axarr[i][j])
            elif 'bar' in plot_type:
                sns.barplot(x='quality',y=column_names[counter],data=dataset, ax=axarr[i][j])
               
            counter+=1
            if counter==(number_of_column-1,):
                break

In [ ]:
data["quality"].value_counts()

In [ ]:
data["quality"].value_counts()

In [ ]:
draw_multivarient_plot(data,4,3,"box")


In [ ]:
data["quality"].value_counts()

In [ ]:

data.drop(['quality'], inplace=True, axis=1)
data.head()

test_df.drop(['quality'], inplace=True, axis=1)
test_df.head()

Lets now find the correlation of the features using a heatmap:

bins = (pd.IntervalIndex.from_tuples([(2,5),(5,8)]))
gnames = ['bad', 'nice']
y = pd.cut(y, bins = bins, labels = gnames)

bins = (pd.IntervalIndex.from_tuples([(2,5),(5,8)]))
gnames = ['bad', 'nice']
y_test = pd.cut(y_test, bins = bins, labels = gnames)

# **3 den 8 e kadar olan kategorileri iyi ve kötü olarak ayırma**

In [ ]:
bins = (2,5,8)
gnames = ['bad', 'nice']
y = pd.cut(y, bins = bins, labels = gnames)

bins = (2,5,8)
gnames = ['bad', 'nice']
y_test = pd.cut(y_test, bins = bins, labels = gnames)


****

# **LabelEncoder ile etiketli sayısallaştırma**

In [ ]:
enc = LabelEncoder()
yenc = enc.fit_transform(y)

enc2 = LabelEncoder()
yenc2 = enc2.fit_transform(y_test)

Using the train_test_split function we divide the labels and data into training and testing data:

In [ ]:
yenc

**verileri ve etiketleri atıyoruz random state i jupyter notebookta yaptığımız için burada ki tek işlemimiz uygun verileri uygun değişkenlere atamak **

In [ ]:
X_train=data
y_train=yenc
X_test=test_df
y_test=yenc2

# **Özellik ölçeklendirme-StandardScaler**

In [ ]:
scale = StandardScaler()

scaledtrain = scale.fit_transform(X_train)
scaledtest = scale.transform(X_test)

Standardizasyon, ortalama değerin 0, standart sapmanın ise 1 değerini aldığı, dağılımın normale yaklaştığı bir metoddur. Formülü şu şekildedir, elimizdeki değerden ortalama değeri çıkartıyoruz, sonrasında varyans değerine bölüyoruz.

# **Model Kurma**

In [ ]:
model = Sequential()

model.add(Dense(128, input_shape=(11,)))
model.add(Activation('relu'))


model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.25))


model.add(Dense(1, activation = 'sigmoid'))


# **Model Compile ve Fit etme**

In [ ]:
rms = Adam(lr=0.0003,decay=1e-6)
#opt = keras.optimizers.Adam(learning_rate=0.0003,decay=1e-6)
#opt2 = keras.optimizers.RMSprop(learning_rate=0.0001)
#opt3 = keras.optimizers.RMSprop(lr=0.00001, decay=1e-6)

model.compile(optimizer=rms, loss='binary_crossentropy', metrics=['accuracy'])
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=1, mode='auto')


In [ ]:
H = model.fit(scaledtrain,
                       y_train, 
                       epochs=100,
                       batch_size = 16,
                       validation_data=(scaledtest,y_test),
                       callbacks=[early_stop]
                       #steps_per_epoch=len(scaledtrain) // 16,
                       #validation_steps=len(X_test) // 16
                       )
   

projemiz süresince val_loss'u düşürmek için fazladan çaba harcadık. Batch bormalizasyonun yanında Dropout gibi klasik yöntemlerin dışında veriyi 2 boyutlu hale getirerek Conv1d, maxpooling ve flatten gibi genellikle görüntü işlemede kullanılan yöntemleri de denedik. Araştırdığımız örnekler arasında en iyiler %80 ile %85 arasındaydı.Bizim modelimizin test başarım oranı bu noktalara geldiğinde model kurma aşamasını bitirmiş olduk

# **Modeli değerlendirme**

In [ ]:
N = 100
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")

In [ ]:
preds=model.predict_classes(scaledtest)

In [ ]:
accuracy_score(y_true=y_test,y_pred=preds)